In [125]:
import pandas as pd
import numpy as np
import gurobipy as gb
from gurobipy import GRB


In [126]:
#load all datasets from the excel files

production_cost = pd.read_csv('Datasets/Production cost of generating units.csv')
start_up_cost = pd.read_csv('Datasets/Start-up cost of generating units.csv')
demand = pd.read_csv('Datasets/Loads.csv')
B = pd.read_csv('Datasets/B (power transfer factor of each bus to each line).csv', sep= ';')
line_capacity = pd.read_csv('Datasets/Transmission capacity of lines.csv')
ramping_rate_gen = pd.read_csv('Datasets/Ramping rate of generating units.csv')
wind_capacity = pd.read_csv('Datasets/Capacity of wind farms.csv')
max_prod_limit = pd.read_csv('Datasets/Maximum production of generating units.csv')
min_prod_limit = pd.read_csv('Datasets/Minimum production of generating units.csv')
min_up_time = pd.read_csv('Datasets/Minimum up time of generating units.csv')
min_down_time = pd.read_csv('Datasets/Minimum down time of generating units.csv')

In [127]:
# define length of dataset
conventional_gen = ['G1', 'G2', 'G3']
wind_gen = ['W1', 'W2']
load_demand = ['D1', 'D2', 'D3']
lines = ['L1', 'L2', 'L3', 'L4', 'L5', 'L6', 'L7']
buses = B.columns

In [128]:
production_cost.index = conventional_gen
start_up_cost.index = conventional_gen
max_prod_limit.index = conventional_gen
min_prod_limit.index = conventional_gen
min_up_time.index = conventional_gen
min_down_time.index = conventional_gen
ramping_rate_gen.index = conventional_gen
demand.index = load_demand
wind_capacity.index = wind_gen
line_capacity.index = lines
B.index = lines

In [129]:
# define the network topology - connect lines to bus
Time = list(range(0, 24))

gen_to_bus = {'G1': 'Bus1','G2': 'Bus2','G3': 'Bus6','W1': 'Bus4','W2': 'Bus5'}
bus_to_gen = {'Bus1': 'G1','Bus2': 'G2','Bus6': 'G3','Bus4': 'W1','Bus5': 'W2'}
demand_to_bus = {'D1': 'Bus4','D2': 'Bus5','D3': 'Bus6'}
bus_to_demand = {'Bus4': 'D1', 'Bus5': 'D2', 'Bus6': 'D3'}

In [130]:
#define the optimization problem
model = gb.Model('Unit Commitment Problem')

In [131]:
#define variables
power_gen = { (g, t): model.addVar(lb = 0, name = f"Power generated in Gen {g} at time {t}")
             for g in conventional_gen + wind_gen
             for t in Time  
            }

commitment = { (g, t): model.addVar(vtype= GRB.BINARY, name = f"Commitment of Gen {g} at time {t}")
              for g in conventional_gen + wind_gen
             for t in Time
             }

start_up_gen_cost = { (g, t): model.addVar(lb = 0, name = f"Start up cost of Gen {g} at time {t}")
                    for g in conventional_gen
                    for t in Time
                    }


In [132]:
for g in wind_gen:
    production_cost.loc[g] = 0
# define the objective function
cost1 =  gb.quicksum(production_cost['cost_op'][g] * power_gen[g, t] for g in conventional_gen + wind_gen for t in Time)
cost2 = gb.quicksum(start_up_gen_cost[g, t] for g in conventional_gen for t in Time)
model.setObjective(cost1 + cost2, GRB.MINIMIZE)


In [133]:
# define the constraints
# constraint 1: Conventional generation within minimun and maximum limits
model.addConstrs((power_gen[g, t] >=  commitment[g, t] * min_prod_limit['pgmin'][g])
                 for g in conventional_gen 
                 for t in Time)

model.addConstrs((power_gen[g, t] <= commitment[g, t] * max_prod_limit['pgmax'][g])
                 for g in conventional_gen 
                 for t in Time)

model.addConstrs((power_gen[g, t] <= commitment[g, t] * wind_capacity['wind'][g])
                 for g in wind_gen 
                 for t in Time)

# constraint 2: power balance equation
for t in Time:
    model.addConstr(gb.quicksum(power_gen[g, t] for g in conventional_gen + wind_gen) 
                == gb.quicksum(demand['load'][l] for l in load_demand),
                name=f"PowerBalance_t{t}")

#constraint 3: Minimum and maximum line capacity limits are enforced
# Initialize net injection dictionary
net_injection = {}

for n in buses:
    for t in Time:
        gen_sum = gb.quicksum(power_gen[g, t] for g in conventional_gen + wind_gen if gen_to_bus[g] == n)
        
        demand_sum = 0
        for d in load_demand:
            if demand_to_bus[d] == n:
                demand_sum += demand['load'][d]
        
        # Net injection at bus n, time t
        net_injection[n, t] = gen_sum - demand_sum

for l in lines:
    for t in Time:
        flow = gb.quicksum(B[n][l]  * net_injection[n, t] for n in buses)
        
        model.addConstr(flow <= line_capacity['fmax'][l], name=f"Flow_upper_L{l}_T{t}")
        
        model.addConstr(flow >= -line_capacity['fmax'][l], name=f"Flow_lower_L{l}_T{t}")

#constraint 4: Start cost of generator is considered if started within the hour
for g in conventional_gen:
    commitment[g, -1] = 0

model.addConstrs(start_up_gen_cost[g, t] >= 0
                 for g in conventional_gen 
                 for t in Time)

for g in conventional_gen:
    for t in Time:
        if t == 0:
            model.addConstr(
                start_up_gen_cost[g, t] >= start_up_cost['cost_st'][g] * (commitment[g, t] - 0),
                name=f"StartUpCost_{g}_T{t}"
            )
        else:
            model.addConstr(
                start_up_gen_cost[g, t] >= start_up_cost['cost_st'][g] * (commitment[g, t] - commitment[g, t-1]),
                name=f"StartUpCost_{g}_T{t}"
            )


model.update()

In [ ]:
#Not sure of this logic, from chatgpt - might need to revisit the constraints for min start up and down time
#constraint on ramp up and down is missing

# Define start-up and shut-down variables
startup = {}
shutdown = {}

initial_commitment = {g: 0 for g in conventional_gen}  # Assume generators are off initially

for g in conventional_gen:
    for t in Time:
        startup[g, t] = model.addVar(vtype=GRB.BINARY, name=f"Startup_{g}_{t}")
        shutdown[g, t] = model.addVar(vtype=GRB.BINARY, name=f"Shutdown_{g}_{t}")

# Link start-up and shut-down variables with commitment variables
for g in conventional_gen:
    for t in Time:
        if t == 0:
            model.addConstr(
                startup[g, t] >= commitment[g, t] - initial_commitment[g],
                name=f"StartupDef_{g}_{t}"
            )
            model.addConstr(
                shutdown[g, t] >= initial_commitment[g] - commitment[g, t],
                name=f"ShutdownDef_{g}_{t}"
            )
        else:
            model.addConstr(
                startup[g, t] >= commitment[g, t] - commitment[g, t - 1],
                name=f"StartupDef_{g}_{t}"
            )
            model.addConstr(
                shutdown[g, t] >= commitment[g, t - 1] - commitment[g, t],
                name=f"ShutdownDef_{g}_{t}"
            )

# Implement minimum up time constraints
for g in conventional_gen:
    UT = int(min_up_time['lu'][g])
    for t in Time:
        if t <= len(Time) - UT:
            model.addConstr(
                gb.quicksum(
                    (1 - commitment[g, k]) for k in range(t, t + UT)
                ) <= (1 - startup[g, t]) * UT,
                name=f"MinUpTime_{g}_{t}"
            )

# Implement minimum down time constraints
for g in conventional_gen:
    DT = int(min_down_time['ld'][g])
    for t in Time:
        if t <= len(Time) - DT:
            model.addConstr(
                gb.quicksum(
                    commitment[g, k] for k in range(t, t + DT)
                ) <= (1 - shutdown[g, t]) * DT,
                name=f"MinDownTime_{g}_{t}"
            )


In [ ]:
# Optimize the model
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-12500H, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 975 rows, 456 columns and 3022 nonzeros
Model fingerprint: 0x5df0c114
Variable types: 192 continuous, 264 integer (264 binary)
Coefficient statistics:
  Matrix range     [7e-02, 9e+02]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Presolve removed 970 rows and 450 columns
Presolve time: 0.03s
Presolved: 5 rows, 6 columns, 17 nonzeros
Variable types: 5 continuous, 1 integer (1 binary)
Found heuristic solution: objective 113907.14162

Root relaxation: objective 1.135674e+05, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*  

In [136]:

print("\nObjective Value:", model.ObjVal)

print("\nPower Generation Values:")
for t in Time:
    print(f"Time {t}:")
    for g in conventional_gen + wind_gen:
        value = power_gen[g, t].X
        print(f"  Power Generation of {g}: {value}")

print("\nCommitment Variables:")
for t in Time:
    print(f"Time {t}:")
    for g in conventional_gen + wind_gen:
        value = commitment[g, t].X
        print(f"  Commitment of {g}: {value}")

print("\nStart-Up Costs:")
for t in Time:
    print(f"Time {t}:")
    for g in conventional_gen:
        value = start_up_gen_cost[g, t].X
        print(f"  Start-Up Cost of {g}: {value}")



Objective Value: 113567.41158997829

Power Generation Values:
Time 0:
  Power Generation of G1: 100.0
  Power Generation of G2: 95.3506444968756
  Power Generation of G3: 44.64935550312439
  Power Generation of W1: 30.0
  Power Generation of W2: 10.0
Time 1:
  Power Generation of G1: 100.0
  Power Generation of G2: 95.3506444968756
  Power Generation of G3: 44.64935550312439
  Power Generation of W1: 30.0
  Power Generation of W2: 10.0
Time 2:
  Power Generation of G1: 100.0
  Power Generation of G2: 95.3506444968756
  Power Generation of G3: 44.64935550312439
  Power Generation of W1: 30.0
  Power Generation of W2: 10.0
Time 3:
  Power Generation of G1: 100.0
  Power Generation of G2: 95.3506444968756
  Power Generation of G3: 44.64935550312439
  Power Generation of W1: 30.0
  Power Generation of W2: 10.0
Time 4:
  Power Generation of G1: 100.0
  Power Generation of G2: 95.3506444968756
  Power Generation of G3: 44.64935550312439
  Power Generation of W1: 30.0
  Power Generation of W

In [124]:
#load all datasets from the excel files

print('\nproduction_cost = \n', production_cost , '\nstart_up_cost= \n',start_up_cost, '\ndemand= \n',demand, '\nB = ',B, '\nline_capacity=\n',line_capacity, '\nramping_rate_gen=\n', ramping_rate_gen, '\nwind_capacity= \n',wind_capacity, '\nmax_prod_limit=\n',max_prod_limit, '\nmin_prod_limit=\n',min_prod_limit, '\nmin_up_time=\n',min_up_time, '\nmin_down_time=\n', min_down_time)


production_cost = 
     cost_op
G1   20.655
G2   16.700
G3   22.545
W1    0.000
W2    0.000 
start_up_cost= 
     cost_st
G1      900
G2      550
G3      170 
demand= 
     load
D1    56
D2   112
D3   112 
B =      Bus1      Bus2      Bus3      Bus4      Bus5      Bus6
L1     0 -0.681967 -0.650181 -0.482661 -0.514447 -0.634718
L2     0  0.146043 -0.753114 -0.221642 -0.322485 -0.704055
L3     0 -0.318033 -0.349819 -0.517339 -0.485553 -0.365282
L4     0  0.171990  0.102932 -0.261020 -0.191962  0.069337
L5     0 -0.146043 -0.246886  0.221642 -0.677515 -0.295945
L6     0 -0.146043 -0.246886  0.221642  0.322485 -0.295945
L7     0  0.146043  0.246886 -0.221642 -0.322485 -0.704055 
line_capacity=
     fmax
L1   200
L2   100
L3   100
L4   100
L5   100
L6   100
L7   100 
ramping_rate_gen=
     ru
G1  55
G2  50
G3  20 
wind_capacity= 
     wind
W1    30
W2    10 
max_prod_limit=
     pgmax
G1    220
G2    100
G3     70 
min_prod_limit=
     pgmin
G1    100
G2     10
G3     10 
min_up_time=
    